In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import json

In [3]:
with open('../utils.json', 'r') as f:    
    json_data = json.load(f)
    file_path = json_data['data']['path']

with h5py.File(file_path, 'r') as h5f:
    traces = h5f['traces'][()]
    plaintexts = h5f['plaintext'][()]
    ciphertexts = h5f['ciphertext'][()]
    keys = h5f['keys'][()]

### Differential Power Analysis (DPA)

$$ \triangle D[j] = \frac{\sum^{m}_{i=1}D(C_i,b,K_s)}{} $$